# Exploremos las capacidades de LangChain
Un framework para desarrollar aplicaciones basadas en LLMs 

Para empezar, debemos primero instalar LangChain. Este proyecto usa `poetry` como el administrador de paquetes. 

In [ ]:
!pip add langchain

##  Ejemplo de diseño y ejecución de Chains o Cadenas 
Las cadenas, son secuencias de llamados a componentes, módulos, procesos o tareas independientes, que en conjunto, son capaces de atacar problemas complejos 

Antes de iniciar, importamos las librerías necesarias para la ejecución de nuestra cadena.

In [ ]:
import os # Utilidades varias del sistema operativo
from dotenv import load_dotenv # Esta librería nos permite cargar las variables de ambiente en memoria
load_dotenv() # Realizamos la carga de las variables de ambiente

## Objetivo de nuestra cadena

Queremos que la inteligencia artificial nos ayude a crear empresas de tecnología. Para esto tenemos que solicitar a la inteligencia artificial la ejecucion de tres tareas:

1. identificar oportunidades de negocio
2. generar el nombre de la empresa
3. Y crear un plan de acción para iniciar su desarrollo

Como fase preparatoria, procedemos a inicializar la conexión con nuestro modelo de lenguaje

In [ ]:
from langchain.chat_models import ChatOpenAI # Nuestro LLM de preferencia


# Inicialización del LLM
llm = ChatOpenAI(
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name='gpt-4-1106-preview',
    temperature = 0.7,
)


Ahora, damos inicio a la creacion de nuestros prompts. Crearemos tres prompts, cada uno preguntando por una parte importante de nuestro proceso de creacion de empresas.

In [ ]:
from langchain.prompts import ChatPromptTemplate # Utilizada para creación de plantillas de prompts de chat


# Identificar las oportunidades de negocio en el sector de interés del usuario
primer_prompt = ChatPromptTemplate.from_template("""Eres un sistema que ayuda a emprendedores de Colombia a generar ideas innovadoras de negocio.
Mi sector de interés es {sector}, y busco una idea que sea un producto, servicio, o una combinación de ambos. 
Solicito que la idea tenga potencial de alto retorno de inversión y cree un impacto significativo (económico, social, ambiental, etc.).
También incluye una descripción del público objetivo de la idea.
Por favor, incluye una breve evaluación de viabilidad y ejemplos o casos de estudio relevantes. 
Busco una respuesta concisa, limitada a un párrafo
Mi futuro profesional depende de ti.
""")

In [ ]:
from langchain.prompts import ChatPromptTemplate # Utilizada para creación de plantillas de prompts de chat


# 2. generar el nombre de la empresa
segundo_prompt = ChatPromptTemplate.from_template("""Eres un sistema que ayuda a emprendedores de Colombia a generar ideas innovadoras de negocio. 
Mi sector de interés es {sector}, y la idea de negocio que tengo implica:

{idea}

Busco un nombre para la empresa que sea atractivo, y que refleje perfecto para la audiencia objetivo. 
Por favor, proporciona la mejor opción de nombre que se cumpla estos criterios.
El nombre debe ser conciso y tu respuesta debe estar limitada a solo el nombre candidato de la compañía.
No justifiques tu respuesta.
Mi futuro profesional depende de ti.
""")

In [ ]:
from langchain.prompts import ChatPromptTemplate # Utilizada para creación de plantillas de prompts de chat


# 3. Y crear un plan de acción para iniciar su desarrollo
tercer_prompt = ChatPromptTemplate.from_template("""Eres un sistema que ayuda a emprendedores a generar ideas innovadoras de negocio. 
Mi sector de interés es {sector}, y la idea de negocio que tengo implica:

{idea}. 

El nombre de mi nuevo emprendimiento es {nombre}. 
Tu tarea es crear un roadmap detallado para el lanzamiento del producto o servicio al mercado. 
Por favor, incluye pasos claros, una línea de tiempo y especifica los entregables de cada paso.
""")

Ahora, inicializamos los eslabones de nuestra cadena y los orquestamos, especificando también los parámetros de entrada y de salida esperados 

In [ ]:
from langchain.chains import LLMChain # Clase LLMChain de LangChain
from langchain.chains import SequentialChain # Clase SimpleSequentialChain de LangChain


primer_eslabon = LLMChain(llm=llm, prompt=primer_prompt, output_key='idea')

segundo_eslabon = LLMChain(llm=llm, prompt=segundo_prompt, output_key='nombre')

tercer_eslabon = LLMChain(llm=llm, prompt=tercer_prompt, output_key='roadmap')

cadena = SequentialChain(
    chains=[primer_eslabon, segundo_eslabon, tercer_eslabon],
    input_variables=['sector'],
    output_variables=['idea', 'nombre', 'roadmap'],
    verbose=True,
)

output = cadena({'sector': 'Inteligencia Artificial'})

Y tras haber realizado la ejecución exitosa, presentamos nuestros resultados

In [ ]:
output

##  Ejemplo de implementación de agentes 

Los `Agentes` tienen la capacidad no solo de ejecutar secuencias de prompts, sino de ejecutar herramientas externas, cuando el LLM ve la necesidad de hacerlo 


### Objetivo de nuestro agente

Queremos que nuestro agente pueda ejecutar la siguiente tarea:

1. Saber la fecha en la que deseamos ir a cine
2. Solicitar la ciudad en la que estamos
3. Recomendarnos la mejor película que está en cartelera
4. Consultar la predicción del clima
5. Realizar una recomendación completa

Inicializamos nuevamente las variables y librerías necesarias para la ejecución exitosa de nuestro agente

In [ ]:
import os # Utilidades varias del sistema operativo
from dotenv import load_dotenv # Esta librería nos permite cargar las variables de ambiente en memoria
load_dotenv() # Realizamos la carga de las variables de ambiente

Creamos las funciones o herramientas que le permitirán al agente ejecutar código local o realizar llamados a servicios remotos 

In [ ]:
from datetime import datetime, timedelta


def traer_fecha(days: str = "0"):
    """Función para traer la fecha de X días hacia adelante"""
    return f'"fecha": {(datetime.today() + timedelta(days=int(days))).strftime("%Y-%m-%d")}'

In [ ]:
import requests
import random


def traer_pelicula(string: str | None = None):
    """Función para traer las películas de un género para un lugar en una fecha especifica"""
    url = f'https://api.themoviedb.org/3/discover/movie'
    params = {
        'language': 'es-ES',
        'include_adult': False,
        'include_video': False, 
        'sort_by': 'popularity.desc',
    }
    headers = {
        "accept": "application/json",
        "Authorization": f'Bearer {os.getenv("TMDB_READ_ACCESS_KEY")}'
    }
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    movies = response.json()['results']
    selected_movie = random.choice(movies)
    return f'"titulo": {selected_movie["original_title"]}, "resumen": {selected_movie["overview"]}'

In [ ]:
import requests
from datetime import date

def traer_prediccion_del_clima(input: str):
    fecha, lugar = input.replace('"', "").split(',')
    """Función para traer la predicción del clima para un lugar en una fecha específica"""
    fecha_obj = f"{fecha} 12:00:00" 
    # Primero se necesita el geocoding de la ciudad
    url = f'http://api.openweathermap.org/geo/1.0/direct'
    params = {
        'appid': os.getenv('OPENWEATHER_API_KEY'),
        'q': lugar,
        'limit': 1,
    }
    lugar_response = requests.get(url, params=params)
    lugar_response.raise_for_status()   
    lat = lugar_response.json()[0]['lat']
    lon = lugar_response.json()[0]['lon']   
    # Ahora podemos solicitar la prediccion del clima para la ubicacion geografica
    url = f'https://api.openweathermap.org/data/2.5/forecast'
    params = {
        'appid': os.getenv('OPENWEATHER_API_KEY'),
        'lat': lat,
        'lon': lon,
        'lang': 'es',
        'units': 'metric',
        'exclude': 'current,minutely,hourly,alerts',
    }   
    response = requests.get(url, params=params)
    response.raise_for_status() 
    # Tomamos el valor del medio día de la fecha solicitada    
    prediccion = [dia for dia in response.json()['list'] if dia['dt_txt'] == f"{fecha} 12:00:00"][0]   
    return f'"temperatura": {prediccion["main"]["temp"]}, "descripcion": {prediccion["weather"][0]["description"]}'

Ahora, empaquetamos las funciones en un array que será comunicado al Agente, notificándole que estas son las herramientas que tiene a su disposición para resolver el problema.

In [ ]:
from langchain.agents import Tool

HERRAMIENTAS = [
  Tool(
    name="TraerFecha",
    func=traer_fecha,
    description="Trae la fecha esperada. Espera un entero especificando un número dado de días hacia adelante de la fecha actual.",
  ),
  Tool(
    name="TraerPelicula",
    func=traer_pelicula,
    description="Traer una recomendación de una película para que el usuario vea. Retorna el título de la película y una breve descripción.",
  ),
  Tool(
    name="TraerPrediccionDelClima",
    func=traer_prediccion_del_clima,
    description="Trae la predicción del clima para la fecha especificada y la ciudad especificada.",
  ),
]

Ahora, preparamos el prompt con ejemplos que le permitan al modelo inferir como orquestar y hacer el uso de herramientas

In [ ]:
AGENTE_FEW_SHOT_EJEMPLOS = [
    """Question: ¿Me ayudas a planear mi salida a cine para ver una película en el cine mañana en Bogotá?
Thought: Necesito encontrar la fecha de mañana. Esto quiere decir que le debo sumar 1 día a la fecha actual
Action: TraerFecha[1]
Observation: "fecha": 2023-11-29
Thought: Necesito recomendar una película en cartelera en Bogotá para la fecha de mañana
Action: TraerPelicula[]
Observation: "titulo": Los Vengadores, "resumen": Una película de superhéroes
Thought: Necesito encontrar la predicción del clima para la fecha 2023-11-29 en Bogota
Action: TraerPrediccionDelClima["2023-11-29", "Bogota"] 
Observation: "temperatura": 20, "descripción": Cielo despejado
Thought: Puedes ver la película - Los Vengadores - que se trata de superhéroes mañana en Bogotá. El clima estará despejado con una temperatura de 20 grados, por lo tanto, asegúrate de usar protector solar.
Action: Finish["Puedes ver la película - Los Vengadores - que se trata de superhéroes mañana en Bogotá. El clima estará despejado con una temperatura de 20 grados, por lo tanto, asegúrate de usar protector solar."]
"""
]

In [ ]:
AGENTE_FEW_SHOT_EJEMPLOS.extend([
    """Question: Quiero ver una película pasado mañana en Bucaramanga. ¿Me ayudas a planear mi salida a cine?
Thought: Necesito encontrar la fecha de mañana. Esto quiere decir que le debo sumar 2 días a la fecha actual
Action: TraerFecha[2]
Observation: "fecha": 2023-11-30
Thought: Necesito recomendar una película de acción en cartelera en Bucaramanga para la fecha de mañana
Action: TraerPelicula[]
Observation: "titulo": Barbie, "resumen": Una película de muñecas, o eso creo
Thought: Necesito encontrar la predicción del clima para la fecha 2023-11-30 en Bucaramanga
Action: TraerPrediccionDelClima["2023-11-30", "Bucaramanga"]
Observation: "temperatura": 27, "descripcion": Lluvioso
Thought: Puedes ver la película - Barbie - que se trata de muñecas o eso creo. El clima estara lluvioso con una temperatura de 27 grados, por lo tanto, asegúrate de llevar sombrilla.
Action: Finish["Puedes ver la película - Barbie - que se trata de muñecas o eso creo. El clima estará lluvioso con una temperatura de 27 grados, por lo tanto, asegúrate de llevar sombrilla."]
""",
    """Pregunta: Oye ayúdame a cuadrar mi salida a cine hoy en Medellín
Thought: Necesito encontrar la fecha de hoy. Esto quiere decir que no le debo sumar días a la fecha actual
Action: TraerFecha[0]
Observation: "fecha": 2023-11-28
Thought: Necesito recomendar una película de acción en cartelera en Medellín para la fecha de hoy
Action: TraerPelicula[]
Observation: "titulo": Oppenheimer, "resumen": Una película de científicos locos
Thought: Necesito encontrar la predicción del clima para la fecha 2023-11-28 en Medellín
Action: TraerPrediccionDelClima["2023-11-28", "Medellín"]
Observation: "temperatura": 24, "descripción": Soleado
Thought: Puedes ver la película - Oppenheimer - que se trata científicos locos. El clima estará soleado con una temperatura de 24 grados, por lo tanto, recuerda el protector solar.
Action: Finish["Puedes ver la película - Oppenheimer - que se trata científicos locos. El clima estará soleado con una temperatura de 24 grados, por lo tanto, recuerda el protector solar."]
""",
])

Creamos un sufijo, dando a grandes rasgos instrucciones del uso de las acciones

In [ ]:
SUFIJO = """\nEres un sistema inteligente realizando una serie de pensamientos y ejecutando acciones para poder responder la pregunta del usuario.
Cada acción es una llamada a una función: TraerFecha(dias: int): str, TraerPelicula(): dict[str, str] y TraerPrediccionDelClima(fecha: str, lugar: str): dict[str, str] 
Por favor, entrega la respuesta sin usar caracteres que puedan causar problemas de parsing como comillas dobles o comillas simples o comas.
Puedes usar la función cuando consideres necesario. Cada acción se realiza por separado.

Vamos a empezar

Question: {input}
{agent_scratchpad}"""


Y organizamos nuestro prompt, utilizando los ejemplos y el sufijo

In [ ]:
from langchain.prompts.prompt import PromptTemplate


PROMPT_AGENTE = PromptTemplate.from_examples(
  examples=AGENTE_FEW_SHOT_EJEMPLOS,
  suffix=SUFIJO,
  input_variables=["input", "agent_scratchpad"],
)

Ahora, creamos nuestro propio agente, customizándolo para que pueda entender los prompts y ejemplos en nuestro idioma

In [ ]:
from typing import Sequence, Any
from langchain.agents.agent import Agent, AgentOutputParser
from langchain.agents.react.output_parser import ReActOutputParser
from langchain.tools.base import BaseTool
from langchain.schema.prompt_template import BasePromptTemplate


class ReActAgent(Agent):
  """
  Agente customizado para el caso de uso de la implementación de la estrategia ReAct
  """

  @classmethod
  def _get_default_output_parser(cls, **kwargs: Any) -> AgentOutputParser:
    return ReActOutputParser()

  @classmethod
  def create_prompt(cls, tools: Sequence[BaseTool]) -> BasePromptTemplate:
    return PROMPT_AGENTE

  @classmethod
  def _validate_tools(cls, tools: Sequence[BaseTool]) -> None:
    if len(tools) != 3:
      raise ValueError("The number of tools is invalid.")

  @property
  def _agent_type(self) -> str:
    return "react"

  @property
  def finish_tool_name(self) -> str:
    return "Finish"

  @property
  def observation_prefix(self) -> str:
    return f"Observation: "

  @property
  def llm_prefix(self) -> str:
    return f"Thought: "

Ahora, tenemos todo listo para invocar a nuestro agente

In [ ]:
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI # Nuestro LLM de preferencia


# Inicializamos nuestro LLM de preferencia
llm = ChatOpenAI(
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name='gpt-4-1106-preview',
    temperature = 0.7,
)

# Creamos una instancia de nuestro agente
agent = ReActAgent.from_llm_and_tools(
  llm,
  HERRAMIENTAS,
)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent,
  tools=HERRAMIENTAS,
  verbose=True,
  handle_parsing_errors=True,
)

In [ ]:
question = "Por fa, quiero ir a cine mañana en Bucaramanga. ¿Me ayudas a planear mi salida?"
agent_executor.run(question)